In [1]:
import pandas as pd
from elasticsearch import Elasticsearch
from src.matomo_connector import MatomoSQLConnector
from dotenv import load_dotenv
import os

load_dotenv()

True

In [2]:
es_connection = Elasticsearch(
            [os.getenv('ELASTICSEARCH_SEARCH_ENGINE_HOST')],            
            api_key=os.getenv('ELASTICSEARCH_SEARCH_ENGINE_API_KEY'),
        )
'OK' if es_connection.ping() else 'KO'

'OK'

In [10]:
matomo = MatomoSQLConnector()
await matomo.connect()

In [14]:
import spacy 

nlp_fr = spacy.load("fr_core_news_md")

import unicodedata

def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    return u"".join([c for c in nfkd_form if not unicodedata.combining(c)])

def preprocess(fulltext: str):
    tokens = nlp_fr(fulltext)
    res = []
    for token in tokens:
        if not token.is_punct and not token.is_space :
            res.append(token.lemma_.lower())
    #misspelled = spell.unknown(res)
    corrected = ' '.join(res)
    # if misspelled :
    #     for word in misspelled :
    #         correction = spell.correction(word)
    #         if correction :
    #             print(word, correction)
    #             corrected = corrected.replace(word, correction)
    return remove_accents(corrected)

/Users/remi/.pyenv/versions/3.12.10/envs/play/lib/python3.12/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'fr_core_news_md' (3.8.0) was trained with spaCy v3.8.0 and may not be 100% compatible with the current version (3.7.5). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [5]:
# themes

res = es_connection.search(
    index="cdtn-preprod-v2_documents",
    size=1000,
    _source=['text'],
    query= {
      "bool": {
        "filter": [
          { "term": { "source": "themes" } },          
        ],
      },    
  });

themes = [t['_source']['text'] for t in res['hits']['hits']]

# themes = (
#     pd.read_csv("themes.csv")["Thème"]
#     .apply(lambda t: t.replace("-", "").strip())
#     .drop_duplicates()
#     .tolist()
# )

In [155]:
# popular keywords


In [11]:
query =  """
        SELECT idVisit, action_id, sitesearchkeyword FROM matomo_partitioned
        WHERE action_type = 'search'
          AND action_timestamp >= '2025-10-21 00:00:00'
          AND action_timestamp < '2025-11-20 00:00:00'
          ORDER BY idVisit
          ;
    """
res = await matomo.run_query(query)

In [12]:
unique_queries = pd.DataFrame(res, columns=['idVisit', 'action_id', 'query']).drop_duplicates(subset=['idVisit', 'query'])

In [15]:
unique_queries['norm'] = unique_queries['query'].apply(preprocess)

In [16]:
norm_counts = pd.DataFrame(unique_queries['norm'].value_counts()).reset_index()

In [17]:
popular_queries = set(norm_counts[norm_counts['count'] > 20]['norm'])

In [18]:
# filter on selected norm counts
unique_queries['selected'] = unique_queries['norm'].apply(lambda q : q in popular_queries)

In [19]:
norm_popular_queries = []
# retrieve the most frequent normalized entry
for key, group in unique_queries[unique_queries['selected']].groupby(['norm']):
    norm_popular_queries.append(group['query'].value_counts().index[0].capitalize())
    

In [20]:
# conventions collectives
res = es_connection.search(
    index="cdtn-preprod-v2_documents",
    size=1000,
    source=[
      "shortTitle",     
    ],
    query= {
      "bool": {
        "filter": [
          { "term": { "source": "conventions_collectives" } },
          { "term": { "isPublished": True } },
          #{ "term": { "contributions": True } },
        ],
      },    
  });

In [152]:
res['hits']['hits'][0]

{'_index': 'cdtn-preprod-v2_documents-1764148853293',
 '_id': '106dd72da8',
 '_score': 0.0,
 '_source': {'shortTitle': 'Vétérinaires : personnel salarié des cabinets et cliniques vétérinaires'}}

In [21]:
conventions = [c['_source']['shortTitle'] for c in res['hits']['hits']]

In [22]:
# existing suggestions

In [23]:
query =  """
        SELECT idVisit, action_id, action_eventaction, action_eventname  FROM matomo_partitioned
        WHERE action_eventcategory = 'selectedSuggestion'
          AND action_timestamp >= '2025-06-01 00:00:00'
          AND action_timestamp < '2025-11-01 00:00:00'
          ORDER BY idVisit
          ;
    """
res = await matomo.run_query(query)


In [24]:
unique_suggestions = pd.DataFrame(res, columns=['idVisit', 'action_id', 'action_eventaction', 'action_eventname']).drop_duplicates(subset=['idVisit', 'action_eventname'])

In [25]:
#unique_suggestions['action_eventname'].value_counts()

In [26]:
sugg_count = pd.DataFrame(unique_suggestions['action_eventname'].value_counts()).reset_index()

In [27]:
popular_suggestions = sugg_count[sugg_count['count'] > 100]['action_eventname'].apply(lambda s : s.capitalize()).tolist()

In [28]:
# for reference : non suggested popular queries
set(unique_queries[unique_queries['selected']]['query']) - set(unique_suggestions['action_eventname'])

{'"fiche de poste"',
 '"temps partiel thérapeutique"',
 '% formation',
 '13eme mois',
 '2216',
 'ABANDON DE POSTE',
 'ACCIDENT DE TRAJET',
 'ACCIDENT DE TRAVAIL',
 'ACCIDENT DU TRAVAIL',
 'AFFICHAGE OBLIGATOIRE',
 'AMPLITUDE ET TEMPS DE PAUSE',
 'ANCIENNETE',
 'APPRENTI',
 'APPRENTISSAGE',
 'ARRET DE TRAVAIL',
 'ARRET MALADIE',
 'ARTICLE DE LOI',
 'ASTREINTE',
 'AVERTISSEMENT',
 'Absences',
 'Accident de travail',
 'Accident du travail',
 'Affichage obligatoire',
 'Allaitement',
 'Ancienneté',
 'Annualisation du temps de travail',
 'Apprentissage',
 'Arret maladie',
 'Arrêt de travail',
 'Arrêt maladie',
 'Assistante maternelle',
 'Avertissement',
 'BDESE',
 'BULLETIN DE PAIE',
 'Bdese',
 'Bulletin de paie',
 'Bulletin de salaire',
 'CDD',
 'CDI',
 'CESU',
 'CLASSIFICATION',
 'CODE DU TRAVAIL',
 'CODIT',
 'COEFFICIENT',
 'CONGE',
 'CONGE ENFANT MALADE',
 'CONGE SANS SOLDE',
 'CONGES',
 'CONGES EXCEPTIONNELS',
 'CONGES SANS SOLDE',
 'CONGÉS',
 'CONGé',
 'CONTRAT CDD',
 'CONTRAT CDI',
 '

In [29]:
new_suggestions = set(themes + norm_popular_queries + conventions + popular_suggestions)

In [30]:
import json
with open("suggestions.txt", "w") as f:
    for entity in new_suggestions:
        f.write(json.dumps({'entity': entity, 'value':42})+'\n')

In [31]:
len(new_suggestions)

646